In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [3]:
# 1. 데이터 로드 및 전처리
data = pd.read_csv('train.csv')
X = data.drop(['Activity', 'subject'], axis=1)
y = pd.get_dummies(data['Activity'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



In [4]:
# 2. XGBoost 모델 훈련
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred))



Validation Accuracy: 0.9750679963735267


In [5]:
# 3. 모델 평가 및 예측
test_data = pd.read_csv('test.csv')
X_test = test_data.drop(['Activity', 'subject'], axis=1)
y_test = pd.get_dummies(test_data['Activity'])
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.8934509670851714


모델의 예측 결과인 y_pred를 기반으로 패턴 밖의 데이터가 들어왔을 때 알림

새로운 데이터를 로드하고 전처리. 이전에 StandardScaler를 사용하여 데이터를 스케일링한 것과 같은 방법으로 스케일링을 해야함.

model.predict() 함수를 사용하여 새로운 데이터의 예측값을 계산.

예측값이 이전에 학습한 클래스 레이블 중 어느 하나와 다르다면, 패턴 밖의 데이터가 들어왔다는 것. 이때 알림을 보내도록 코드를 작성.

In [ ]:
new_data = pd.read_csv('new_data.csv')  # 새로운 데이터를 로드합니다.
new_data = scaler.transform(new_data.drop(['Activity', 'subject'], axis=1))  # 이전에 사용한 스케일러를 사용하여 데이터를 스케일링합니다.
y_new_pred = model.predict(new_data)  # 새로운 데이터의 예측값을 계산합니다.

if not any((y_new_pred == y_train).all(axis=1)):  # 예측값이 이전에 학습한 클래스 레이블 중 어느 하나와 다르다면
    send_alert()  # 알림을 보냅니다.


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# 1. 데이터 로드 및 전처리
data = pd.read_csv('train.csv')
X = data.drop(['Activity', 'subject'], axis=1)
y = pd.get_dummies(data['Activity'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# 2. XGBoost 모델 훈련
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# 3. 모델 평가 및 예측
test_data = pd.read_csv('test.csv')
X_test = test_data.drop(['Activity', 'subject'], axis=1)
y_test = pd.get_dummies(test_data['Activity'])
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)

# 4. 모델 예측 결과 평가
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

# 5. 패턴 밖의 데이터 예측
new_data = pd.read_csv('new_data.csv')
X_new = new_data.drop(['Activity', 'subject'], axis=1)
y_new = pd.get_dummies(new_data['Activity'])
X_new = scaler.transform(X_new)
y_pred_new = model.predict(X_new)

# 6. 패턴 밖의 데이터 예측 결과 평가 및 알림
accuracy_new = accuracy_score(y_new, y_pred_new)
print("New Data Accuracy:", accuracy_new)

threshold = 0.05  # 일정 오차 범위 설정
diff = np.abs(y_new.values.argmax(axis=1) - y_pred_new.argmax(axis=1))
if np.max(diff) <= threshold:
    print("예측 결과가 정확합니다.")
else:
    print("경고: 예측 결과가 오차 범위를 초과합니다!")
